In [ ]:
import os
import shutil
import fastkaggle
from pathlib import Path
import numpy as np
import pandas as pd
import polars as pl
from datasets import Dataset
import torch  # base
import torch.nn.functional as F
import json
from transformers import (
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    AutoTokenizer,
    DataCollatorWithPadding,
)
from sklearn.model_selection import train_test_split

In [ ]:
data_base_path = Path("./data")
comp_name = "llm-classification-finetuning"
datapath = data_base_path / comp_name
if not os.path.exists(datapath) and not datapath.exists():
    install_path = fastkaggle.setup_comp(comp_name)
    shutil.move(install_path, datapath)

In [ ]:
pl.Config.set_tbl_width_chars(300)
pl.Config.set_fmt_str_lengths(300)

polars.config.Config

In [ ]:
df_sample = pl.read_csv(datapath / "sample_submission.csv")
df_sample.head()

id,winner_model_a,winner_model_b,winner_tie
i64,f64,f64,f64
136060,0.333333,0.333333,0.333333
211333,0.333333,0.333333,0.333333
1233961,0.333333,0.333333,0.333333


In [ ]:
df_train = pl.read_csv(datapath / "train.csv")
df_test: pl.DataFrame = pl.read_csv(datapath / "test.csv")
df_train.head()

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
i64,str,str,str,str,str,i64,i64,i64
30192,"""gpt-4-1106-preview""","""gpt-4-0613""","""[""Is it morally right to try to have a certain percentage of females on managerial positions?"",""OK, does pineapple belong on a pizza? Relax and give me fun answer.""]""","""[""The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\n\nHere are some arguments in favor of and against such policies:\n\n**Argume…","""[""As an AI, I don't have personal beliefs or opinions. However, I can tell you that the question of gender quotas in managerial positions is a complex one and positions can vary. Advocates argue that such measures can help correct historical imbalances, promote diversity, and may lead to better deci…",1,0,0
53567,"""koala-13b""","""gpt-4-0613""","""[""What is the difference between marriage license and marriage certificate?"",""How can I get both of them as quick as possible in California "",""What is the minimal time to get them? 1 day or 10 days?""]""","""[""A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually one year. After the marriage has taken place, the couple must obtain a marriage…","""[""A marriage license and a marriage certificate are two different legal documents that have separate purposes.\n\n1. Marriage License: A marriage license is a legal document that gives a couple permission to get married. It's usually obtained from local government or court officials before the weddi…",0,1,0
65089,"""gpt-3.5-turbo-0613""","""mistral-medium""","""[""explain function calling. how would you call a function?""]""","""[""Function calling is the process of invoking or executing a function in a programming language. To call a function, you need to use the function's name followed by parentheses \""()\"". Within the parentheses, you can pass arguments if the function requires any.\n\nHere is an example of how to call a…","""[""Function calling is the process of invoking a function in a program. A function is a block of code that performs a specific task and can be reused multiple times in a program. Functions help to organize and modularize code, making it more readable and maintainable.\n\nTo call a function, you simpl…",0,0,1
96401,"""llama-2-13b-chat""","""mistral-7b-instruct""","""[""How can I create a test set for a very rare category? I want to build a classifier.""]""","""[""Creating a test set for a very rare category can be challenging, as it may not be possible to collect a sufficient number of examples for the rare category. However, there are several strategies you can use to create a test set for a rare category:\n\n1. Use synthetic data: If the rare category is…","""[""When building a classifier for a very rare category, creating a test set can be challenging due to the limited amount of data available. Here are some steps you can take to create a test set for a rare category:\n\n1. Gather as much data as possible: The more data you have, the easier it will be t…",1,0,0
198779,"""koala-13b""","""gpt-3.5-turbo-0314""","""[""What is the best way to travel from Tel-Aviv to Jerusalem? Car? Bus? Plane?""]""","""[""The best way to travel from Tel Aviv to Jerusalem depends on your personal preference and the availability of transportation options. All the options you have mentioned are valid options, but here are some details to help you make your decision:\n\n* By car: Traveling by car is the quickest way …","""[""The best way to travel from Tel-Aviv to Jerusalem depends on your personal preferences and circumstances. \n\nIf you have a car and enjoy driving, then driving from Tel-Aviv to Jerusalem is an optio

In [ ]:
df_train["prompt"][0]

'["Is it morally right to try to have a certain percentage of females on managerial positions?","OK, does pineapple belong on a pizza? Relax and give me fun answer."]'

: 

: 

TODO:

- Split multi-turn columns to new rows
  - ~~ensure len(resp) matches len(answer)~~
  - ~~If any are not lists, handle~~
  - ~~Remove `[]`~~
  - ~~use id as user_id or convo_id~~
  - ^ nevermind, thanks polars!
  - preprocess


Split each user/convo into a row


In [ ]:
cols = ["prompt", "response_a", "response_b"]
if isinstance(df_train["prompt"][0], str):
    first_prompt_cell = df_train["prompt"][0]
    if "[" in first_prompt_cell and "]" in first_prompt_cell:
        df_train = df_train.with_columns([pl.col(col).str.json_decode() for col in cols])
        df_train = df_train.explode(cols)

In [ ]:
df_train.head()

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie
i64,str,str,str,str,str,i64,i64,i64
30192,"""gpt-4-1106-preview""","""gpt-4-0613""","""Is it morally right to try to have a certain percentage of females on managerial positions?""","""The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination. Here are some arguments in favor of and against such policies: **Arguments in…","""As an AI, I don't have personal beliefs or opinions. However, I can tell you that the question of gender quotas in managerial positions is a complex one and positions can vary. Advocates argue that such measures can help correct historical imbalances, promote diversity, and may lead to better decisi…",1,0,0
30192,"""gpt-4-1106-preview""","""gpt-4-0613""","""OK, does pineapple belong on a pizza? Relax and give me fun answer.""","""Ah, the age-old culinary conundrum that has divided nations and dinner tables: does pineapple belong on a pizza? The tropical twist of pineapple on pizza, known as Hawaiian pizza, is a hotly debated topic where taste buds battle and pizza purists protest. Let's slice into the debate with a zest of …","""As an AI, I don't eat, so I don't have personal preferences. But, in the world of pizza, the pineapple topping is a hotly debated topic! Some people love the sweet and savory mix it brings to a pizza, a flavor combination made famous by the Hawaiian pizza. Others firmly believe that the sweetness of…",1,0,0
53567,"""koala-13b""","""gpt-4-0613""","""What is the difference between marriage license and marriage certificate?""","""A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually one year. After the marriage has taken place, the couple must obtain a marriage c…","""A marriage license and a marriage certificate are two different legal documents that have separate purposes. 1. Marriage License: A marriage license is a legal document that gives a couple permission to get married. It's usually obtained from local government or court officials before the wedding c…",0,1,0
53567,"""koala-13b""","""gpt-4-0613""","""How can I get both of them as quick as possible in California ""","""If you want to get both a marriage license and a marriage certificate in California as quickly as possible, there are a few steps you can follow: 1. Obtain a marriage license: You can obtain a marriage license from the county clerk's office in the county where you plan to marry. The process typic…","""In California, here are the general steps to obtain a marriage license and marriage certificate as quickly as possible: 1. Marriage License: - Both parties must appear together at one of the County Clerk’s offices. - Complete a marriage license application before arriving (available online i…",0,1,0
53567,"""koala-13b""","""gpt-4-0613""","""What is the minimal time to get them? 1 day or 10 days?""","""The minimal time to get a marriage license and a marriage certificate in California can vary depending on the specific requirements and process in your county. Generally, you can obtain a marriage license and a marriage certificate in California within a few days of applying, but it is best to check…","""The marriage license in California is issued immediately when you apply in person and meet all the requirements, so theoretically, you could get married on the same day if you have an officiant ready to perform the ceremony. However, the marriage certificate, which is the official record of your ma…",0,1,0


In [ ]:
def null_clean(df: pl.DataFrame):
    df = df.drop_nulls(subset=["prompt", "response_a", "response_b"])
    return df

In [ ]:
df_train = null_clean(df_train)
df_test = null_clean(df_test)

In [ ]:
(df_train["prompt"] + " [SEP] " + df_train["response_a"] + " [SEP] " + df_train["response_b"])[0]

'Is it morally right to try to have a certain percentage of females on managerial positions? [SEP] The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination.\n\nHere are some arguments in favor of and against such policies:\n\n**Arguments in favor:**\n\n1. **Correcting Historical Inequities:** Women have historically been underrepresented in leadership roles due to various cultural, institutional, and social barriers. Aiming for a specific percentage can be seen as a corrective measure to address past and ongoing discrimination.\n\n2. **Promoting Diversity:** Diverse leadership teams can enhance decision-making and represent a broader range of perspectives. This can lead to better outcomes for organizations and society as a whole.\n\n3. **Equality of Opportunity:** Setting targets for female representation in management can he

In [ ]:
def apply_bert_fmt(df: pl.DataFrame):
    df = df.with_columns(
        (
            pl.col("prompt") + " [SEP] " + pl.col("response_a") + " [SEP] " + pl.col("response_b")
        ).alias("text")
    )
    return df

Condense to a single target column and remap to 0,1,2 as possible dep vars


In [ ]:
label_map = {
    "model_a": 0,
    "model_b": 1,
    "tie": 2,
}
df_train = df_train.with_columns(
    pl.when(pl.col("winner_model_a") == 1)
    .then(pl.lit(0))
    .when(pl.col("winner_model_b") == 1)
    .then(pl.lit(1))
    .when(pl.col("winner_tie") == 1)
    .then(pl.lit(2))
    .otherwise(pl.lit(None))
    .alias("label")
)

In [ ]:
df_train = apply_bert_fmt(df_train)
df_test = apply_bert_fmt(df_test)

In [ ]:
df_train.head()

id,model_a,model_b,prompt,response_a,response_b,winner_model_a,winner_model_b,winner_tie,label,text
i64,str,str,str,str,str,i64,i64,i64,i32,str
30192,"""gpt-4-1106-preview""","""gpt-4-0613""","""Is it morally right to try to have a certain percentage of females on managerial positions?""","""The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, and discrimination. Here are some arguments in favor of and against such policies: **Arguments in…","""As an AI, I don't have personal beliefs or opinions. However, I can tell you that the question of gender quotas in managerial positions is a complex one and positions can vary. Advocates argue that such measures can help correct historical imbalances, promote diversity, and may lead to better decisi…",1,0,0,0,"""Is it morally right to try to have a certain percentage of females on managerial positions? [SEP] The question of whether it is morally right to aim for a certain percentage of females in managerial positions is a complex ethical issue that involves considerations of fairness, equality, diversity, a…"
30192,"""gpt-4-1106-preview""","""gpt-4-0613""","""OK, does pineapple belong on a pizza? Relax and give me fun answer.""","""Ah, the age-old culinary conundrum that has divided nations and dinner tables: does pineapple belong on a pizza? The tropical twist of pineapple on pizza, known as Hawaiian pizza, is a hotly debated topic where taste buds battle and pizza purists protest. Let's slice into the debate with a zest of …","""As an AI, I don't eat, so I don't have personal preferences. But, in the world of pizza, the pineapple topping is a hotly debated topic! Some people love the sweet and savory mix it brings to a pizza, a flavor combination made famous by the Hawaiian pizza. Others firmly believe that the sweetness of…",1,0,0,0,"""OK, does pineapple belong on a pizza? Relax and give me fun answer. [SEP] Ah, the age-old culinary conundrum that has divided nations and dinner tables: does pineapple belong on a pizza? The tropical twist of pineapple on pizza, known as Hawaiian pizza, is a hotly debated topic where taste buds batt…"
53567,"""koala-13b""","""gpt-4-0613""","""What is the difference between marriage license and marriage certificate?""","""A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually one year. After the marriage has taken place, the couple must obtain a marriage c…","""A marriage license and a marriage certificate are two different legal documents that have separate purposes. 1. Marriage License: A marriage license is a legal document that gives a couple permission to get married. It's usually obtained from local government or court officials before the wedding c…",0,1,0,1,"""What is the difference between marriage license and marriage certificate? [SEP] A marriage license is a legal document that allows a couple to get married. It is issued by a government agency, such as a county clerk's office or a state government, and is valid for a certain period of time, usually o…"
53567,"""koala-13b""","""gpt-4-0613""","""How can I get both of them as quick as possible in California ""","""If you want to get both a marriage license and a marriage certificate in California as quickly as possible, there are a few steps you can follow: 1. Obtain a marriage license: You can obtain a marriage license from the county clerk's office in the county where you plan to marry. The process typic…","""In California, here are the general steps to obtain a marriage license and marriage certificate as quickly as possible: 1. Marriage License: - Both parties must appear together at one of the County Clerk’s offices. - Complete a marriage license application before arriving (availab

In [ ]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    df_train["text"], df_train["label"], test_size=0.1, random_state=42
)

In [ ]:
model_name = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
train_texts[0]

'Al Jazeera is from where? [SEP] Al Jazeera is a prominent global media network originally from Doha, the capital city of Qatar. It was launched on November 1, 1996, by the Emir of Qatar at the time, Sheikh Hamad bin Khalifa Al Thani. The name "Al Jazeera" translates to "The Peninsula" in English, referring to the Arabian Peninsula. Al Jazeera is known for its independent and often critical coverage of news and current affairs, both within the Middle East and around the world. [SEP] Al Jazeera is a media network based in Doha, Qatar. It was launched in 1996 and has grown to be a major global news organization, with multiple TV channels and digital platforms providing news coverage in Arabic, English, and other languages. Though it is owned by the government of Qatar, Al Jazeera sees itself as an independent news source providing objective journalism from an Arab and broader global south perspective. Its headquarters and original Arabic language channel are located in Doha.'

In [ ]:
ex_enc = tokenizer(train_texts[0], truncation=True, padding=True)
ex_enc

{'input_ids': [101, 2632, 14855, 23940, 2527, 2003, 2013, 2073, 1029, 102, 2632, 14855, 23940, 2527, 2003, 1037, 4069, 3795, 2865, 2897, 2761, 2013, 26528, 1010, 1996, 3007, 2103, 1997, 12577, 1012, 2009, 2001, 3390, 2006, 2281, 1015, 1010, 2727, 1010, 2011, 1996, 23434, 1997, 12577, 2012, 1996, 2051, 1010, 12840, 10654, 4215, 8026, 27925, 2632, 2084, 2072, 1012, 1996, 2171, 1000, 2632, 14855, 23940, 2527, 1000, 16315, 2000, 1000, 1996, 6000, 1000, 1999, 2394, 1010, 7727, 2000, 1996, 13771, 6000, 1012, 2632, 14855, 23940, 2527, 2003, 2124, 2005, 2049, 2981, 1998, 2411, 4187, 6325, 1997, 2739, 1998, 2783, 3821, 1010, 2119, 2306, 1996, 2690, 2264, 1998, 2105, 1996, 2088, 1012, 102, 2632, 14855, 23940, 2527, 2003, 1037, 2865, 2897, 2241, 1999, 26528, 1010, 12577, 1012, 2009, 2001, 3390, 1999, 2727, 1998, 2038, 4961, 2000, 2022, 1037, 2350, 3795, 2739, 3029, 1010, 2007, 3674, 2694, 6833, 1998, 3617, 7248, 4346, 2739, 6325, 1999, 5640, 1010, 2394, 1010, 1998, 2060, 4155, 1012, 2295, 2009, 2

In [ ]:
train_encodings = tokenizer(list(train_texts), truncation=True, padding=True)
val_encodings = tokenizer(list(val_texts), truncation=True, padding=True)
test_encodings = tokenizer(list(df_test["text"]), truncation=True, padding=True)

In [ ]:
train_encodings.attention_mask[0]

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [ ]:
ex_enc.input_ids

[101,
 2632,
 14855,
 23940,
 2527,
 2003,
 2013,
 2073,
 1029,
 102,
 2632,
 14855,
 23940,
 2527,
 2003,
 1037,
 4069,
 3795,
 2865,
 2897,
 2761,
 2013,
 26528,
 1010,
 1996,
 3007,
 2103,
 1997,
 12577,
 1012,
 2009,
 2001,
 3390,
 2006,
 2281,
 1015,
 1010,
 2727,
 1010,
 2011,
 1996,
 23434,
 1997,
 12577,
 2012,
 1996,
 2051,
 1010,
 12840,
 10654,
 4215,
 8026,
 27925,
 2632,
 2084,
 2072,
 1012,
 1996,
 2171,
 1000,
 2632,
 14855,
 23940,
 2527,
 1000,
 16315,
 2000,
 1000,
 1996,
 6000,
 1000,
 1999,
 2394,
 1010,
 7727,
 2000,
 1996,
 13771,
 6000,
 1012,
 2632,
 14855,
 23940,
 2527,
 2003,
 2124,
 2005,
 2049,
 2981,
 1998,
 2411,
 4187,
 6325,
 1997,
 2739,
 1998,
 2783,
 3821,
 1010,
 2119,
 2306,
 1996,
 2690,
 2264,
 1998,
 2105,
 1996,
 2088,
 1012,
 102,
 2632,
 14855,
 23940,
 2527,
 2003,
 1037,
 2865,
 2897,
 2241,
 1999,
 26528,
 1010,
 12577,
 1012,
 2009,
 2001,
 3390,
 1999,
 2727,
 1998,
 2038,
 4961,
 2000,
 2022,
 1037,
 2350,
 3795,
 2739,
 3029,
 1010,
 2

In [ ]:
train_encodings.input_ids[0] == ex_enc.input_ids

False